# NamedTuple
https://docs.python.org/3/library/collections.html#collections.namedtuple

As simple as tuple, but with better readability.

**Tuple**

In [1]:
x=1
y=2
point=(x,y)
point

(1, 2)

In [2]:
point[0]

1

In [3]:
point[1]

2

**NamedTuple**

In [4]:
from collections import namedtuple
Point=namedtuple("Point","x,y", defaults=[-1,-2])

x=1
y=2
point=Point(x,y)
point

Point(x=1, y=2)

In [5]:
point.x

1

In [6]:
point.y

2

In [7]:
point.y = 5

AttributeError: can't set attribute

**Useful helper functions**  
(namedtuple methods start with single underscore. They are not private.)

In [8]:
point._replace(x=5)

Point(x=5, y=2)

In [9]:
point._asdict()

{'x': 1, 'y': 2}

In [10]:
point._fields

('x', 'y')

In [11]:
point._field_defaults

{'x': -1, 'y': -2}

In [12]:
Point()

Point(x=-1, y=-2)

# Dataclass
https://docs.python.org/3/library/dataclasses.html  
https://pyvideo.org/pycon-us-2018/dataclasses-the-code-generator-to-end-all-code-generators.html  
Not a replacement for namedtuple. Code generator to reduce common boilerplate.

Can be customized in many ways to do what we want.


In [13]:
from dataclasses import dataclass, field

@dataclass
class Point:
    x: int
    y: int

Point(1,2)


Point(x=1, y=2)

In [14]:
@dataclass
class Point:
    x: int = -1
    y: int = -2

Point(y=6)

Point(x=-1, y=6)

In [27]:
@dataclass
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6, repr=False) # Will not be in autogenerated repr

Point(1,2)

Point(x=1)

In [28]:
Point(1,2).y

2

In [16]:
@dataclass(repr=False) # Does not generate repr, so the dataclass inherits objects default repr
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

Point(1,2)

In [17]:
@dataclass(init=False) # __init__ method will not be generated
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

Point(1,2)

TypeError: Point() takes no arguments

In [18]:
point= Point(1,2)
point.x = 0

TypeError: Point() takes no arguments

**Dataclasses in sets**

In [19]:
@dataclass
class FrozenPoint:
    x: int = field(default=-1)
    y: int = field(default=-6)

{FrozenPoint(1,2), FrozenPoint(1,1)}

TypeError: unhashable type: 'FrozenPoint'

In [20]:
frozen_point= FrozenPoint(1,2)
frozen_point.x = 6

In [21]:
@dataclass(frozen=True)
class FrozenPoint:
    x: int = field(default=-1)
    y: int = field(default=-6)

{FrozenPoint(1,2), FrozenPoint(1,1)}


{FrozenPoint(x=1, y=1), FrozenPoint(x=1, y=2)}

In [22]:
{FrozenPoint(1,1), FrozenPoint(1,1)}


{FrozenPoint(x=1, y=1)}

**Dataclasses in comparisons**

In [23]:
@dataclass()
class Point:
    x: int = field(default=-1)
    y: int = field(default=-6)

Point(1,2) == Point(1,2)

True

In [24]:
@dataclass()
class Point:
    x: int = field(default=-1, compare=False)
    y: int = field(default=-6)

Point(666, 2) == Point(1,2)

True

In [25]:
Point(1,666) == Point(1,2)

False

In [26]:
{Point(666, 2), Point(1,2)}

TypeError: unhashable type: 'Point'

In [67]:
@dataclass(frozen=True)
class Point:
    x: int = field(default=-1, compare=False)
    y: int = field(default=-6)
{Point(666, 2), Point(1,2)}

{Point(x=666, y=2)}